# Domain, Halo and Padding regions

Now we will learn how to build up expressions using the `Halo` and `Padding` regions. For that, we will use the time marching example shown in [01_iet](https://github.com/opesci/devito/blob/master/examples/compiler/01_iet.ipynb) tutorial. 

In [1]:
from devito import Eq, Grid, TimeFunction, Operator

grid = Grid(shape=(3, 3))
u = TimeFunction(name='u', grid=grid)
u.data[:] = 1

At this moment, we have a time-varying 3x3 grid filled with `1's`. Below, we can see the `domain` data values:

In [2]:
print(u.data)

[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


We now create an `Operator` that increments by `2` all points in the computational domain at each timestep.

In [3]:
eq = Eq(u.forward, u+2)
op = Operator(eq)

Finally, we can print the `op` to see the generated code.

In [4]:
print(op)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct profiler
{
  double section0;
} ;


int Kernel(float *restrict u_vec, const int time_M, const int time_m, struct profiler* timers, const int x_M, const int x_m, const int x_size, const int y_M, const int y_m, const int y_size)
{
  float (*restrict u)[x_size + 1 + 1][y_size + 1 + 1] __attribute__((aligned(64))) = (float (*)[x_size + 1 + 1][y_size + 1 + 1]) u_vec;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
  {
    struct timeval start_section0, end_section0;
    gettimeofday(&start_section0, NULL);
    for (int x = x_m; x <= x_M; x += 1)
    {
      #pragma omp simd
      for (int y = y_m; y <= y_M; y += 1)
    

When we take a look at the constructed expression ($u[t1][x + 1][y + 1] = u[t0][x + 1][y + 1] + 2$) we see `+1` being added to the spatial indexes of `u`.

That occurs because we have grid points surrounding the `domain region`, i.e. ghost points that are accessed by the stencil when iterating in the proximity of the domain boundary. This region is called `Halo`. The Halo region can be seen below, it's the zeros surrounding the domain region.

In [5]:
print(u.data_with_halo)

[[[0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0.]
  [0. 1. 1. 1. 0.]
  [0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0.]
  [0. 1. 1. 1. 0.]
  [0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0.]]]


Thus, the array accesses become logically aligned to the equation’s natural domain. For instance, given the usual Function $u(t, x, y)$ having one point on each side of the `x` and `y` halo regions, the array accesses $u[t, x, y]$ and $u[t, x + 2, y + 2]$ are transformed, respectively, into $u[t, x + 1, y + 1]$ and $u[t, x + 3, y + 3]$. When $x = y = 0$, therefore, the values $u[t, 1, 1]$ and $u[t, 3, 3]$ are fetched, representing the first and third points in the computational domain. 

By default, the Halo region has `1` point on each side of the space dimensions. Sometimes, those points may be unnecessary. On the other hand, for instance, depending on the PDE being approximated, more points may be necessary. Thus, this default value can be changed by passing a value in `space_order`:

In [6]:
u0 = TimeFunction(name='u0', grid=grid, space_order=0)
u0.data[:] = 1
print(u0.data_with_halo)

[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


In [7]:
u2 = TimeFunction(name='u2', grid=grid, space_order=2)
u2.data[:] = 1
print(u2.data_with_halo)

[[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]]


Also, one can pass a 3-tuple `(o, lp, rp)` instead of a single integer representing the discretization order. Here, `o` is the discretization order, while `lp` and `rp` indicate how many points are expected on left (lp) and right (rp) of a point of interest.

In [8]:
u_new = TimeFunction(name='u_new', grid=grid, space_order=(4, 3, 1))

In [9]:
u_new.data[:] = 1
print(u_new.data_with_halo)

[[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 1. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]]


Let's write the code again, but, this time, changing the space_order values.

In [10]:
equation = Eq(u_new.forward, u_new + 2)
op = Operator(equation)
print(op)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct profiler
{
  double section0;
} ;


int Kernel(float *restrict u_new_vec, const int time_M, const int time_m, struct profiler* timers, const int x_M, const int x_m, const int x_size, const int y_M, const int y_m, const int y_size)
{
  float (*restrict u_new)[x_size + 1 + 3][y_size + 1 + 3] __attribute__((aligned(64))) = (float (*)[x_size + 1 + 3][y_size + 1 + 3]) u_new_vec;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
  {
    struct timeval start_section0, end_section0;
    gettimeofday(&start_section0, NULL);
    for (int x = x_m; x <= x_M; x += 1)
    {
      #pragma omp simd
      for (int y = y_m; y <= y_M; 

Finally, let's run the operator.

In [11]:
op.apply(time_M=2)
print(u_new.data_with_halo)

Operator `Kernel` run in 0.00 s


[[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 5. 5. 5. 0.]
  [0. 0. 0. 5. 5. 5. 0.]
  [0. 0. 0. 5. 5. 5. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 7. 7. 7. 0.]
  [0. 0. 0. 7. 7. 7. 0.]
  [0. 0. 0. 7. 7. 7. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]]


There is a region that surrounding the halo region, that region is named `padding`. Padding can be used for data alignment. By default there is no padding, but it can be changed by passing a value in `padding`, let's see below:

In [12]:
u_pad = TimeFunction(name='u_pad', grid=grid, space_order=2, padding=(0,2,2))
u_pad.data_allocated[:] = 3
u_pad.data_with_halo[:] = 2
u_pad.data[:] = 1
equation = Eq(u_pad.forward, u_pad + 2)
op = Operator(equation)
print(op)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"

struct profiler
{
  double section0;
} ;


int Kernel(float *restrict u_pad_vec, const int time_M, const int time_m, struct profiler* timers, const int x_M, const int x_m, const int x_size, const int y_M, const int y_m, const int y_size)
{
  float (*restrict u_pad)[x_size + 2 + 2 + 2 + 2][y_size + 2 + 2 + 2 + 2] __attribute__((aligned(64))) = (float (*)[x_size + 2 + 2 + 2 + 2][y_size + 2 + 2 + 2 + 2]) u_pad_vec;
  /* Flush denormal numbers to zero in hardware */
  _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
  _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
  for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
  {
    struct timeval start_section0, end_section0;
    gettimeofday(&start_section0, NULL);
    for (int x = x_m; x <= x_M; x += 1)
    {
      #pragma omp simd
  

Now we see `+4` being added on each side of the space dimensions. Of which, +2 belong to space_order (halo) and +2 belong to padding.
We can see the data using `data_allocated`.

In [13]:
print(u_pad.data_allocated)

[[[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
  [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 2. 2. 1. 1. 1. 2. 2. 3. 3.]
  [3. 3. 2. 2. 1. 1. 1. 2. 2. 3. 3.]
  [3. 3. 2. 2. 1. 1. 1. 2. 2. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
  [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]]

 [[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
  [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 2. 2. 1. 1. 1. 2. 2. 3. 3.]
  [3. 3. 2. 2. 1. 1. 1. 2. 2. 3. 3.]
  [3. 3. 2. 2. 1. 1. 1. 2. 2. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
  [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
  [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]]]


The `data_allocated` function show `domain + halo + padding` data values. Above, the domain is filled with 1's, the halo is filled with 2's and the padding is filled with 3's.